In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import upload_data, pull_model, pull_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get model from our Hopsworks model registry

In [3]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Get the full Ethereum data from Hopsworks feature store.

In [4]:
eth_ohlc = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)

eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (2.60s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,2015-11-15 00:00:00+00:00,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,0,1,0,0,0,0,0,0,1
1224,2015-11-16 00:00:00+00:00,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,0,1,1,0,0,0,0,0,0
2431,2015-11-17 00:00:00+00:00,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,0,1,0,1,0,0,0,0,0
1967,2015-11-18 00:00:00+00:00,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,0,1,0,0,1,0,0,0,0
859,2015-11-19 00:00:00+00:00,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,2024-04-18 00:00:00+00:00,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,0,0,0,0,0,0,1,0,0,0
2792,2024-04-19 00:00:00+00:00,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,0,0,0,0,0,0,0,1,0,0
2704,2024-04-20 00:00:00+00:00,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3158.908566,0.001237,...,0,0,0,0,0,0,0,0,1,0
3062,2024-04-21 00:00:00+00:00,3142.949775,3170.679379,3137.747159,3147.137754,9.264213e+09,2.940255e+06,3.778353e+11,3215.276150,0.021192,...,0,0,0,0,0,0,0,0,0,1


In [5]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_ohlc[predictors]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,0,1
1224,1.012313,NaN,1.012313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,1,0,0,0,0,0,0
2431,1.031419,100.000000,1.041774,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0,0,1,0,1,0,0,0,0,0
1967,1.003638,69.765684,0.990000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,1,0,0,1,0,0,0,0
859,0.989331,34.147244,0.982267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,1.004777,55.265104,1.013228,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0
2792,1.003858,64.503124,1.003415,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
2704,1.008724,85.064674,1.011218,2.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3062,1.002061,75.270475,0.998752,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
x0['last_100_day_1_0_percent_increase_count']

806      NaN
1224     NaN
2431     NaN
1967     NaN
859      NaN
        ... 
2750    51.0
2792    51.0
2704    51.0
3062    51.0
3061    51.0
Name: last_100_day_1_0_percent_increase_count, Length: 3063, dtype: float64

In [9]:
predictions = model.predict(x0)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [8]:
len(prediction)

3063

In [ ]:
pred = int(prediction[0])
eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = pred

eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = eth_yesterday['pred_tmw_1_0_percent_increase_binary'].astype('Int64')